# Breed classifcation

This notebook lists all the steps that you need to complete this project. You will need to complete all the TODOs in this notebook as well as in the README and the two python scripts included with the starter code.


**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of these the TODO's and use more than one TODO code cell to do all your tasks.

In [2]:
# TODO: Install any packages that you might need
# For instance, you will need the smdebug package
!pip install smdebug

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 3.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.3/83.3 kB 925.5 kB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.9 MB/s eta 0:00:00:00:01
  Created wheel for pyinstrument-cext: filename=pyinstrument_cext-0.2.4-cp310-cp310-linux_x86_64.whl size=6415 sha256=5fd5a1ad44cd2db8cca3c36a753fe656eef368f86698007a84b05d367e8f8ff5
  Stored in directory: /root/.cache/pip/wheels/0f/8b/7a/5f7fd1dd6d3cbb3d350d4c832c5e2f962687749f6d67d573a6
Successfully built pyinstrument-cext
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.3
    Uninstalling protobuf-4.25.3:
      Successfully uninstalled protobuf-4.25.3


In [36]:
import os
import csv
import io
import numpy as np
import pandas as pd
from pprint import pprint
from PIL import Image

import boto3
from botocore.exceptions import NoCredentialsError

import sagemaker
from sagemaker.pytorch import PyTorch, PyTorchModel
from sagemaker.debugger import (
    DebuggerHookConfig,
    Rule,
    ProfilerRule,
    rule_configs,
    TensorBoardOutputConfig,
    ProfilerConfig
)

## Dataset

For this project, we are using the Udacity Dog Breed Identification dataset. This dataset contains images of dogs categorized into different breeds and is used for training machine learning models to identify dog breeds from images.

In [8]:
%%capture
# Command to download and unzip data
#!wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
#!unzip dogImages.zip

In [37]:
# Initialize S3 client
s3 = boto3.client('s3')

In [47]:
# Define the default bucket
default_bucket = sagemaker.Session().default_bucket()
print(default_bucket)

sagemaker-us-east-1-482545180177


In [39]:
# Define the bucket name and data directory
data_directory = 'dogImages'
# List to store the data for the CSV file
data_for_csv = []

In [ ]:
# Function to upload a file to S3
def upload_to_aws(local_file, bucket, s3_file):
    try:
        s3.upload_file(local_file, bucket, s3_file)
        print(f"Upload Successful: {s3_file}")
        return True
    except FileNotFoundError:
        print("The file was not found")
        return False
    except NoCredentialsError:
        print("Credentials not available")
        return False

In [16]:
%%capture
# Walk through the data directory and upload each file
for root, dirs, files in os.walk(data_directory):
    for file in files:
        local_file = os.path.join(root, file)
        s3_file = os.path.relpath(local_file, data_directory)
        upload_to_aws(local_file, default_bucket, s3_file)

        # Extract type, breed, and dog_image_name from the file path
        parts = os.path.relpath(local_file, data_directory).split(os.sep)
        if len(parts) == 3:  # Ensure the path has exactly 3 parts: type/breed/image
            type_, breed, dog_image_name = parts
            data_for_csv.append({
                'type': type
                'breed': breed,
                'dog_image_name': dog_image_name
            })


In [20]:
# Write the data to a CSV file
csv_file = 'dog_images_info.csv'
csv_columns = ['type', 'breed', 'dog_image_name']

with open(csv_file, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=csv_columns)
    writer.writeheader()
    for row in data_for_csv:
        writer.writerow(row)

print(f"CSV file '{csv_file}' created successfully.")

CSV file 'dog_images_info.csv' created successfully.


In [24]:
# Read the CSV file
csv_file = 'dog_images_info.csv'
df = pd.read_csv(csv_file)

# Display the first few rows of the dataframe
print("First few rows of the dataframe:")
print(df.head())

# Summary analysis
summary = {
    'total_images': len(df),
    'images_per_type': df['type'].value_counts().to_dict(),
    'images_per_breed': df['breed'].value_counts().to_dict(),
    'types_per_breed': df.groupby('breed')['type'].nunique().to_dict()
}

First few rows of the dataframe:
   type                    breed                 dog_image_name
0  test           114.Otterhound           Otterhound_07430.jpg
1  test           114.Otterhound           Otterhound_07405.jpg
2  test           114.Otterhound           Otterhound_07394.jpg
3  test           114.Otterhound           Otterhound_07435.jpg
4  test  110.Norwegian_lundehund  Norwegian_lundehund_07218.jpg


In [33]:
# Prettify the output
print("\nSummary:")
print(f"Total breeds: {len(summary['images_per_breed'])}\n")

print("Images per type:")
for type_, count in summary['images_per_type'].items():
    print(f"  {type_}: {count}")

print("\nImages per breed:")
for breed, count in summary['images_per_breed'].items():
    print(f"  {breed}: {count}")


Summary:
Total breeds: 133

Images per type:
  train: 6680
  test: 836
  valid: 835

Images per breed:
  005.Alaskan_malamute: 96
  029.Border_collie: 93
  015.Basset_hound: 92
  057.Dalmatian: 89
  039.Bull_terrier: 87
  014.Basenji: 86
  041.Bullmastiff: 86
  046.Cavalier_king_charles_spaniel: 84
  012.Australian_shepherd: 83
  011.Australian_cattle_dog: 83
  087.Irish_terrier: 82
  008.American_staffordshire_terrier: 82
  056.Dachshund: 82
  023.Bernese_mountain_dog: 81
  032.Boston_terrier: 81
  036.Briard: 81
  076.Golden_retriever: 80
  044.Cane_corso: 80
  006.American_eskimo_dog: 80
  001.Affenpinscher: 80
  027.Bloodhound: 80
  021.Belgian_sheepdog: 80
  034.Boxer: 80
  115.Papillon: 79
  042.Cairn_terrier: 79
  068.Flat-coated_retriever: 79
  004.Akita: 79
  071.German_shepherd_dog: 78
  020.Belgian_malinois: 78
  051.Chow_chow: 78
  024.Bichon_frise: 77
  017.Bearded_collie: 77
  082.Havanese: 76
  061.English_cocker_spaniel: 76
  060.Dogue_de_bordeaux: 75
  016.Beagle: 74


## Hyperparameter Tuning
**TODO:** This is the part where you will finetune a pretrained model with hyperparameter tuning. Remember that you have to tune a minimum of two hyperparameters. However you are encouraged to tune more. You are also encouraged to explain why you chose to tune those particular hyperparameters and the ranges.

**Note:** You will need to use the `hpo.py` script to perform hyperparameter tuning.

In [40]:
#TODO: Declare your HP ranges, metrics etc.
hyperparameter_ranges = {
    'batch_size':sagemaker.tuner.CategoricalParameter([32, 64, 128, 256, 512]),
    'epochs': sagemaker.tuner.IntegerParameter(5, 50),
    'lr':  sagemaker.tuner.ContinuousParameter(0.0001, 0.1),
    'num_classes': sagemaker.tuner.IntegerParameter(10, 133)
}

objective_metric_name = 'validation:accuracy'
metric_definitions = [{'Name': 'validation:accuracy', 'Regex': 'Accuracy: ([0-9\\.]+)'}]

In [48]:
os.environ["SM_CHANNEL_TRAINING"] = f's3://{default_bucket}/dogImages/'
os.environ['SM_MODEL_DIR'] = f's3://{default_bucket}/model/'

In [49]:
# TODO: Create estimators for your HPs
estimator = PyTorch(
    entry_point='hpo.py',
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type='ml.c4.xlarge',
    framework_version='1.8.1',
    py_version='py3',
    hyperparameters={
        'data_dir': '/opt/ml/input/data/training',
        'model_path': '/opt/ml/model',
    }
)

# Define the input data channels
inputs = {'training': f's3://{default_bucket}/dogImages/train'}

# TODO: Your HP tuner here
tuner = sagemaker.tuner.HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=20,
    max_parallel_jobs=3
)

In [50]:
# TODO: Fit your HP Tuner
tuner.fit(inputs)

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


................................................*


UnexpectedStatusException: Error for HyperParameterTuning job pytorch-training-240804-1152: Failed. Reason: No training job succeeded after 5 attempts. For additional details, please take a look at the training job failures by listing training jobs for the hyperparameter tuning job.

In [ ]:
# TODO: Get the best estimators and the best HPs
best_estimator = tuner.best_estimator()

# Get the hyperparameters of the best trained model
best_hyperparameters = best_estimator.hyperparameters()

# Print the best hyperparameters
print(f"Best hyperparameters: {best_hyperparameters}")

## Model Profiling and Debugging
TODO: Using the best hyperparameters, create and finetune a new model

**Note:** You will need to use the `train_model.py` script to perform model profiling and debugging.

In [ ]:
# TODO: Set up debugging and profiling rules and hooks
# Set up debugging and profiling rules and hooks
debugger_rules = [
    Rule.sagemaker(rule_configs.loss_not_decreasing()),
    Rule.sagemaker(rule_configs.overtraining()),
]

profiler_rules = [
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
]

profiler_config = ProfilerConfig(
    framework_profile_params=FrameworkProfiler(
        start_step=5,
        num_steps=10
    )
)

debugger_hook_config = DebuggerHookConfig(
    s3_output_path=f's3://{default_bucket}/dogImages/train'
)

In [ ]:
# Create and fit an estimator
estimator = PyTorch(
    entry_point='train_model.py',
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type='ml.c4.xlarge',
    framework_version='1.8.1',
    py_version='py3',
    hyperparameters=best_hyperparameters,
    debugger_hook_config=debugger_hook_config,
    profiler_config=profiler_config,
    rules=debugger_rules + profiler_rules
)

# Define the input data channels
inputs = {'training': f's3://{default_bucket}/dogImages/train'}

# Fit the estimator
estimator.fit(inputs)

In [ ]:
# TODO: Plot a debugging output.
tensorboard_output_config = TensorBoardOutputConfig(
    s3_output_path=f's3://{default_bucket}/tensorboard-output'
)

# Assuming that you have enabled tensorboard in your train_model.py script
estimator.fit(inputs, tensorboard_output_config=tensorboard_output_config)

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [ ]:
# TODO: Display the profiler output
profiler_report_s3_uri = f's3://{default_bucket}/profiler-output/{estimator.latest_training_job.name}/profiler-output/profiler-report.html'
print(f'Profiler report: {profiler_report_s3_uri}')

## Model Deploying

In [ ]:
# TODO: Deploy your model to an endpoint
predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',
    endpoint_name='dog-breed-classifier-endpoint'
)

# Function to preprocess the image
def preprocess_image(image_path):
    from torchvision import transforms

    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    image = Image.open(image_path).convert("RGB")
    image = preprocess(image)
    image = np.expand_dims(image.numpy(), axis=0)
    return image


In [ ]:
# TODO: Run an prediction on the endpoint
image_path = f's3://{default_bucket}/dogImages/test/Affenpinscher_00003.jpg'  # Path to your test image
image = preprocess_image(image_path)

# Convert the image to the format expected by the model
payload = np.array(image).tolist()
response = predictor.predict(payload)

# Decode the prediction response
predicted_class = np.argmax(response)
print(f'Predicted class: {predicted_class}')

In [ ]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
#predictor.delete_endpoint()